In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import triangle as tr

numeric_tolerance = 1e-8

# Step 0: Loading the enclosed mesh 

In [2]:
# for this project, we use a 2D mesh
meshfile = 'data/man.off'
v, f = igl.read_triangle_mesh(meshfile)
v = v[:, :2]

In [3]:
def normalize_vertices(vertices):
    centroid = np.mean(vertices, axis=0)
    translated_vertices = vertices - centroid
    max_distance = np.max(np.linalg.norm(translated_vertices, axis=1))
    normalized_vertices = translated_vertices / max_distance
    return normalized_vertices

v = normalize_vertices(v)

# Step 1: Create a cage

In [4]:
# Cage Utilities
def rotate_vertices(vertices, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians)],
        [np.sin(angle_radians), np.cos(angle_radians)]
    ])
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    return rotated_vertices

def scale_vertices(vertices, scale_factor):
    scaled_vertices = scale_factor * vertices
    return scaled_vertices

def generate_cage_vertices(num_vertices):
    R = 1 / (2 * np.sin(np.pi / num_vertices))
    angles = np.linspace(0, 2*np.pi, num_vertices, endpoint=False)
    x_coords = R * np.cos(angles)
    y_coords = R * np.sin(angles)
    cage_vertices = np.column_stack((x_coords, y_coords))
    return cage_vertices

def centroid_added_cage(cage_vertices): 
    # add the last, which is the centroid of the other vertices
    centroid = np.mean(cage_vertices, axis=0)
    res = np.vstack((cage_vertices, centroid))    
    return res 

def generate_cage_edges(num_vertices):
    return np.column_stack((np.arange(num_vertices), np.arange(1, num_vertices + 1))) % num_vertices

def generate_cage_faces(num_vertices):
    # [0, 1, n-1], [1, 2, n-1], ..., [n-2, n-1, n-1]
    # where n is the centroid
    n = num_vertices
    faces = [[i, i + 1, n] for i in range(n - 1)]
    faces.append([n - 1, 0, n]) 
    return np.array(faces)
    
    

In [12]:
initial_n = 7
cage_n, cage_rotation, cage_scale = initial_n, 0, 1.5
cage_vertices, cage_edges, cage_faces = generate_cage_vertices(initial_n), generate_cage_edges(initial_n), generate_cage_faces(initial_n)

In [13]:
f.shape, cage_faces.shape

((5040, 3), (7, 3))

In [14]:
# Function to update the plot with new data
def update_plot(cage_vertices, cage_edges, v, f):
    global p

# Function to be called when changed 
def update_selection_value(cage_n_arg, cage_rotation_arg, cage_scale_arg):
    global cage_n, cage_rotation, cage_scale, cage_vertices, cage_edges, v
    cage_n, cage_rotation, cage_scale = cage_n_arg, cage_rotation_arg, cage_scale_arg
    cage_vertices = generate_cage_vertices(cage_n)
    cage_vertices = rotate_vertices(cage_vertices, cage_rotation)
    cage_vertices = scale_vertices(cage_vertices, cage_scale)
    cage_edges = generate_cage_edges(cage_n)
    # v = scale_vertices(v, cage_scale)
    # update_plot(cage_vertices, cage_edges, cage_scale, f)

# Widget Wrapper
def widgets_wrapper():
    value_widget = iw.IntSlider(min=4, max=10, value=cage_n, description="Cage Vertices:")
    cage_rotation_widget = iw.FloatSlider(min=-180, max=180, value=cage_rotation, description="Rotation:")
    cage_scale_widget = iw.FloatSlider(min=0.1, max=2, value=cage_scale, description="Scale:")
    
    def on_change(_):
        update_selection_value(value_widget.value, cage_rotation_widget.value, cage_scale_widget.value)
    
    widgets = [value_widget, cage_rotation_widget, cage_scale_widget]
    for widget in widgets: widget.observe(on_change)
    return iw.VBox(widgets)

Outputs. Change From the Widget, then re-run to see the mesh. 

In [15]:
# Widget UI
iw.interact(widgets_wrapper);

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [20]:
p = mp.plot(centroid_added_cage(cage_vertices), cage_edges, return_plot=True)
p.add_mesh(v, f, c=np.array([1, 0, 0]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0493968…

1

This still needs some work to make it interactive. Right now, it is working, just that it doesn't show live changes until you re-run the widget.

# Step 2: Triangulate the cage and Get Harmonic Weights

In [21]:
# def get_triangulated_cage
cage_dict = dict(vertices=cage_vertices)
cage_triangulated = tr.triangulate(cage_dict, 'qa0.01Y') # we need Y so we don't need to care about constraints betweeen cage vertices
v_C, f_C = cage_triangulated['vertices'], cage_triangulated['triangles']
mp.plot(v_C, f_C, shading={"wireframe": True})
p2 = mp.plot(v_C, f_C, c=np.array([1, 0, 0]), shading={"wireframe": True})
p2.add_mesh(v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0493968…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0493968…

1

In [22]:
# there are n harmonic weights to compute per vertex in `C`
n_weights = cage_vertices.shape[0]
n_c_verts = v_C.shape[0]
h = np.zeros((n_c_verts, n_weights)) # h[a, i] is the `i-th` harmonic weight of the `a-th` vertex in C
for i in range(n_weights): 
    # The boundaries vertices we need to set are the vertices of the cage `C`, and the boundary conditions is that the `i-th` harmonic weight of the `j-th` cage vertex is `1` if `i == j` and `0` otherwise.
    b = np.arange(n_weights, dtype=f_C.dtype) # the first `n` vertices of `C` is the cage vertex
    bc = np.zeros((n_weights, 1))
    bc[i] = 1
    
    # Compute the harmonic weights
    weights = igl.harmonic(v_C, f_C, b, bc, 1)
    h[:, i] = weights
assert(np.sum(h, axis=1).all() == 1)

## The visualization of harmonic weights for each cage vertex over all vertices in S.

In [23]:
p = mp.subplot(v_C, f_C, c=h[:, 0], shading={"wireframe": True}, s=[5,3,0])
for i in range(1, n_weights): 
    mp.subplot(v_C, f_C, h[:, i], shading={"wireframe": True}, s=[5,3,i], data=p)

# Step 3: Get Barycentric-Interpolated Harmonic Weights in Enclosed Mesh

Subgoal: each vertex in `S` - `v_S_l` lies Euclideanly in face `ijk` in `C`. Find barycentric weights of `v_S_l` in terms of `i, j, k`. To do this, we find barycentric weights of `v_S_l` in terms of all vertices in `C` and find where it is positive. 

We represent [[a1, b1], [a2, b2], [a3, b3]] at index `l` as `l` in terms of `i, j, k` as `l = a1*i + b1*j + a2*i + b2*j + a3*i + b3*j`.

In [24]:
# a, b, c are the vertices of the triangles in the cage mesh -- a[i], b[i], c[i] makes the `i-th` triangle
# we want to find where `point` is in the cage mesh and the barycentric weights of `point` in the triangle formed by `a[i], b[i], c[i]`
def point_barycentric_information_in_cage_mesh(point, a, b, c): 
    # a,b,c is a vector of the traingles. we want to find points of face i, j, k in a single entry of a, b, c such that `point` is in the triangle formed by i, j, k
    assert(point.shape == (2,))
    assert(a.shape[0] == b.shape[0] == c.shape[0])
    P = a.shape[0] # documentation uses `P` to denote the number of points
    point_repeated_vec = np.repeat(np.expand_dims(point, axis=0), P, axis=0)
    
    # get barycentric weight of `point` (padded as 3D) in the triangle formed by a, b, c
    def pad_3d(v_2d): return np.column_stack((v_2d, np.zeros(v_2d.shape[0])))
    point_repeated_vec, a, b, c = pad_3d(point_repeated_vec), pad_3d(a), pad_3d(b), pad_3d(c)
    barycentric_weights_vec = igl.barycentric_coordinates_tri(point_repeated_vec, a, b, c)
    positive_condition_on_weights = np.all(barycentric_weights_vec >= 0, axis=1)
    positive_barycentric_indices = np.where(positive_condition_on_weights)[0]
    assert(positive_barycentric_indices.shape[0] <= 1) # otherwise, the point is not in the cage or is in multiple triangles

    # index in to get i, j, k and weights 
    positive_index = positive_barycentric_indices[0]
    barycentric_weights = barycentric_weights_vec[positive_index]
    w_i, w_j, w_k = barycentric_weights
    assert(np.abs(w_i + w_j + w_k - 1) < numeric_tolerance)
    
    return positive_index, w_i, w_j, w_k

In [25]:
n_a_verts = v.shape[0]
a, b, c = v_C[f_C[:, 0], :], v_C[f_C[:, 1], :], v_C[f_C[:, 2], :]
h_p = np.zeros((n_a_verts, n_weights))
for l in range(n_a_verts):
    t, bary_w_i, bary_w_j, bary_w_k = point_barycentric_information_in_cage_mesh(v[l], a, b, c) 
    i, j, k = f_C[t] # get the indices of the vertices of the triangle
    h_p[l, :] = bary_w_i * h[i] + bary_w_j * h[j] + bary_w_k * h[k]
assert(np.sum(h_p, axis=1).all() == 1)
# return h_p

The visualization of harmonic weights for each cage vertex over all vertices in `S`.

In [26]:
p = mp.subplot(v, f, c=h_p[:, 0], shading={"wireframe": True}, s=[5,3,0])
for i in range(1, n_weights): 
    mp.subplot(v, f, h_p[:, i], shading={"wireframe": True}, s=[5,3,i], data=p)

# Step 4: Calculate New Mesh Vertices

In [27]:
# return new vertices
def calculate_deformation(h_p, cage_vertices): 
    # actually, this is as simple as a matrix multiplication, which is represented as a dot product. 
    return np.dot(h_p, cage_vertices)

In [33]:
pos_f_saver = np.zeros((cage_vertices.shape[0], 2))
def pos_f(s,x,y):
    global cage_vertices, v, p
    cage_vertices[s,:] = cage_vertices[s] + np.array([x,y])
    pos_f_saver[s] = [x,y]
    t0 = time.time()
    v = pos_f.deformer(cage_vertices)
    colors = np.array([1.0, 0.0, 0.0])
    # p.update_object(oid = 0, vertices = cage_vertices, colors = colors)
    # p.update_object(oid = 1, vertices = v, faces = f, colors = colors)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x: calculate_deformation(h_p, x)

In [40]:
# Widget Wrapper
def widgets_wrapper():
    cage_widget = iw.Dropdown(options=np.arange(cage_n), description="Cage Vertex:")
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xy'}
    
    def update_seg(*args):
        (translate_widget['x'].value,
        translate_widget['y'].value) = pos_f_saver[cage_widget.value]
        
    cage_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=cage_widget)
    widgets_dict.update(translate_widget)
    return widgets_dict

Once again, first run the widget, do modifications, then re-run the plot in the next cell to see changes. 

In [41]:
# Widget UI
iw.interact(pos_f,
            **widgets_wrapper())

interactive(children=(Dropdown(description='Cage Vertex:', options=(0, 1, 2, 3, 4, 5, 6), value=0), FloatSlide…

<function __main__.pos_f(s, x, y)>

In [43]:
p = mp.plot(centroid_added_cage(cage_vertices), cage_faces, return_plot=True)
p.add_mesh(v, f, shading={"wireframe": True}, c=np.array([0, 0, 1]))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.4092490…

1